In [1]:
from agents import Agent, Runner, ModelSettings
from pathlib import Path
import os, re, asyncio

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or Path("/home/mike/workspace/soa-ilec/soa-ilec/.openai_key").read_text().strip()

In [3]:
class PVCalcAgent:

    def __init__(self, openai_key=OPENAI_API_KEY, timeout = 3600, max_turns=100, model="gpt-5-mini"):
        self.timeout = timeout
        self.max_turns = max_turns
        self.model = model
        os.environ["OPENAI_API_KEY"] = openai_key    

    async def prompt_async(self, prompt):
        
        agent = Agent(
            name=str(self.__class__),
            instructions="Your response should be a single floating point number.",              
            model=self.model,
        )

        result = await Runner.run(agent, str(prompt), max_turns=self.max_turns)
        
        return result.final_output

    def prompt(self, prompt: str) -> str:
        try:
            _ = asyncio.get_running_loop()
        except RuntimeError:
            # No running loop: safe to use asyncio.run directly
            return asyncio.run(self.prompt_async(prompt))
        else:
            # A loop is already running (e.g., Jupyter). Run in a separate thread.
            import concurrent.futures
            with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
                fut = ex.submit(lambda: asyncio.run(self.prompt_async(prompt)))
                return fut.result()


In [4]:
PV_CALC_CSV = """
t,amt,1-year rate @ t
1,900,0.05
2,944,0.04
3,580,0.03
4,946,0.05
5,689,0.01
6,996,0.01
7,850,0.05
8,796,0.01
9,919,0.03
10,652,0.03
11,881,0.03
12,767,0.05
13,689,0.05
14,864,0.03
15,844,0.02
16,987,0.02
17,762,0.04
18,937,0.01
19,816,0.04
20,538,0.02
21,516,0.01
22,592,0.05
23,799,0.05
24,814,0.02
25,555,0.01
26,997,0.04
27,528,0.01
28,673,0.01
29,699,0.01
30,978,0.04""".strip()

In [5]:
pv_agent = PVCalcAgent(OPENAI_API_KEY)

In [6]:
res = pv_agent.prompt("Calculate the present value of the following cash flows using the following data:\n" + PV_CALC_CSV)
res

'15241.68'

In [7]:
pv_values = [float(res)]

In [8]:
N_TRIES = 20

while len(pv_values) < N_TRIES:
    print("running...")
    res = pv_agent.prompt("Calculate the present value of the following cash flows using the following data:\n" + PV_CALC_CSV)
    try:
        pv_values.append(float(res))
    except (ValueError, TypeError):
        print(f"Invalid response: {res}")
        pv_values.append(0)
    print("done")

running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done


In [9]:
pv_values

[15241.68,
 15256.94,
 15252.649959611723,
 15258.16,
 15251.52,
 15260.29,
 15255.93,
 15257.524024142856,
 15251.323715,
 15258.2016860718,
 15254.587839428572,
 15258.85,
 15252.43,
 15254.43,
 15265.91,
 15234.3,
 15259.86,
 15243.48,
 15256.22,
 15251.655042142856]

In [39]:
import subprocess

mcp_proc = subprocess.Popen(
    ["uvicorn", "--app-dir", ".", "agent_pv_mcp:app", "--host", "127.0.0.1", "--port", "9090"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE)

In [40]:
from agents.mcp import MCPServerStreamableHttp

class MCPToolPVCalcAgent:

    def __init__(self, openai_key=OPENAI_API_KEY, timeout = 3600, max_turns=100, model="gpt-5-mini"):
        self.timeout = timeout
        self.max_turns = max_turns
        self.model = model
        os.environ["OPENAI_API_KEY"] = openai_key    

    async def prompt_async(self, prompt):
        
        async with MCPServerStreamableHttp(
            name="ilec",
            client_session_timeout_seconds = self.timeout,
            params={
                "url": "http://127.0.0.1:9090/mcp/", 
                "timeout": self.timeout
            },
            cache_tools_list=True,
        ) as mcp_server:
        
            agent = Agent(
                name=str(self.__class__),
                instructions="Your response should be a single floating point number.",              
                mcp_servers=[mcp_server],
                model=self.model
            )

            result = await Runner.run(agent, str(prompt), max_turns=self.max_turns)
        
        return result.final_output

    def prompt(self, prompt: str) -> str:
        try:
            _ = asyncio.get_running_loop()
        except RuntimeError:
            # No running loop: safe to use asyncio.run directly
            return asyncio.run(self.prompt_async(prompt))
        else:
            # A loop is already running (e.g., Jupyter). Run in a separate thread.
            import concurrent.futures
            with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
                fut = ex.submit(lambda: asyncio.run(self.prompt_async(prompt)))
                return fut.result()


In [41]:
mcp_pv_agent = MCPToolPVCalcAgent(OPENAI_API_KEY)

In [42]:
res = mcp_pv_agent.prompt("Calculate the present value of the following cash flows using the following data:\n" + PV_CALC_CSV)
res

'15259.86732764119'

In [43]:
mcp_pv_values = [float(res)]


In [44]:
N_TRIES = 20

while len(mcp_pv_values) < N_TRIES:
    print("running...")
    res = mcp_pv_agent.prompt("Calculate the present value of the following cash flows using the following data:\n" + PV_CALC_CSV)
    try:
        mcp_pv_values.append(float(res))
    except (ValueError, TypeError):
        print(f"Invalid response: {res}")
        mcp_pv_values.append(0)
    print("done")

running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done
running...
done


In [ ]:
np.min(mcp_pv_values)

np.float64(15259.86732764119)

In [46]:
np.max(mcp_pv_values)

np.float64(15259.86732764119)

In [47]:
np.std(mcp_pv_values)

np.float64(0.0)